In [58]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urlencode
import urllib3
import string

In [36]:
http = urllib3.PoolManager()

In [124]:
url = "http://mjl.clarivate.com/cgi-bin/jrnlst/jlresults.cgi?"

In [127]:
alphabets = list(string.ascii_uppercase)

In [157]:
df = pd.DataFrame(columns={'journals'}) 

for alp in log_progress(alphabets):
    for no in range(1,50):
        req = http.request('GET', url + "PC=T&Alpha={ALP}&Page={NO}".format(ALP=alp, NO=no))
        doc = bs(req.data, 'html.parser')
        
        journals_temp = doc.select("#results ul li h4")
        
        if journals_temp != []:
            journals_temp = list(map(lambda x: x.text, journals_temp))
            temp_df = pd.DataFrame({"journals":journals_temp})
            
            df = df.append(temp_df, ignore_index=True)
            
        else:
            continue

VBox(children=(HTML(value=''), IntProgress(value=0, max=26)))

In [159]:
df.to_pickle("journals.p")

In [147]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )